In [2]:
!pip install -q lightgbm tsfresh optuna scikit-learn pandas numpy matplotlib seaborn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 8.1 MB/s eta 0:00:00


In [8]:
import os
import numpy
import pandas
import joblib
import optuna
import tsfresh
import sklearn
import seaborn
import lightgbm
import matplotlib
from google.colab import drive


def EDA(file):
  pandas.set_option('display.max_columns', 200)
  target_col = 'target'
  print("Shape from file: ", file.shape)
  # display(file.head())
  # print(file.dtypes)

def PipelineCreation(file, target_col):
  from sklearn.model_selection import train_test_split
  from sklearn.pipeline import Pipeline
  from sklearn.impute import SimpleImputer
  from sklearn.preprocessing import OneHotEncoder
  from sklearn.compose import ColumnTransformer

  id_cols = [c for c in ['id', 'time', 'index'] if c in file.columns]
  x = file.drop(columns=[target_col] + id_cols, errors='ignore')
  y = file[target_col]

  numeric_features = x.select_dtypes(include=['number']).columns.tolist()
  categorical_features = x.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
  # print("Numeric: ", len(numeric_features))
  # print("Categorical: ", len(categorical_features))
  # print("X: ", x)
  # print("Y: ", y)

  numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')),])

  categorical_transformer = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                                      ('ohe', OneHotEncoder(handle_unknown='ignore')),])
  print(f"Num:{len(numeric_features)}")
  preprocessor = ColumnTransformer([
      ('num', numeric_transformer, numeric_features),
      ('cat', categorical_transformer, categorical_features),
  ], remainder='drop')
  return x, y, preprocessor

def Training(x_train_tr, y_train):
  from lightgbm import LGBMClassifier
  scale_pos_weight = (y_train==0).sum() / max(1, (y_train==1).sum())
  model = LGBMClassifier(
      objective='binary',
      n_estimators=10000,
      learning_rate=0.05,
      num_leaves=31,
      random_state=42,
      scale_pos_weight=scale_pos_weight
  )
  model.fit(
      x_train_tr, y_train,
      eval_set=[(x_val_tr, y_val)],
      eval_metric='auc',
      #my version does not support that method. I need to use callbacks
      #early_stopping_rounds=100,
      #callbacks=[early_stopping(100), log_evaluation(100)]
  )
  print("Best iteration:", model.best_iteration_)
  print("Train AUC:", model.best_score_['training']['auc'])
  print("Valid AUC:", model.best_score_['valid_0']['auc'])

  return model



from sklearn.model_selection import train_test_split
if os.path.exists('/content/drive') == 0:
  drive.mount('/content/drive')

labels = pandas.read_csv('/content/drive/MyDrive/AI_assets/labels.csv')
light_curves  = pandas.read_csv('/content/drive/MyDrive/AI_assets/light_curves.csv')
metadata = pandas.read_csv('/content/drive/MyDrive/AI_assets/metadata.csv')
data = pandas.read_csv('/content/drive/MyDrive/data2.csv')

# im gonna change this under cause i nedd more classes. Using binary interpretation insted of 'CONFIRMED' will help a lot
#x, y, preprocessor = PipelineCreation(data, target_col='kepoi_name')

data['target'] = data['koi_disposition'].map(
    lambda v: 1 if v == "CONFIRMED" else 0
)

x, y, preprocessor = PipelineCreation(data, target_col='target')
print("First step done. 1 -> PIPELINE CREATION")
# print("X: ", x)
# print("Y: ", y)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.20, stratify=y, random_state=42
 )
preprocessor.fit(x_train)
x_train_tr = preprocessor.transform(x_train)
x_val_tr = preprocessor.transform(x_val)
EDA(labels)

#debug purposes
print("Second step done. 2 -> EDA DONE")
print("X_train shape:", x_train_tr.shape)
print("X_val shape:", x_val_tr.shape)
print("y_train distribution:\n", y_train.value_counts())
print("y_val distribution:\n", y_val.value_counts())
print("Check for NaNs:", x_train_tr.isna().sum().sum(), "in train,", x_val_tr.isna().sum().sum(), "in val")


#model = Training(x_train_tr, y_train)
#print("Finished Training. 3!!")


# A litle degub made by GPT
# print("Best iteration:", model.best_iteration_)
# print("Train AUC:", model.best_score_['training']['auc'])
# print("Valid AUC:", model.best_score_['valid_0']['auc'])


Num:133
First step done. 1 -> PIPELINE CREATION
Shape from file:  (4, 2)
Second step done. 2 -> EDA DONE
X_train shape: (8, 172)
X_val shape: (2, 172)
y_train distribution:
 target
1    6
0    2
Name: count, dtype: int64
y_val distribution:
 target
0    1
1    1
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['koi_gmag_err' 'koi_rmag_err' 'koi_imag_err' 'koi_zmag_err'
 'koi_kepmag_err' 'koi_model_dof' 'koi_model_chisq' 'koi_eccen_err1'
 'koi_eccen_err2' 'koi_longp' 'koi_longp_err1' 'koi_longp_err2'
 'koi_sma_err1' 'koi_sma_err2' 'koi_ingress' 'koi_ingress_err1'
 'koi_ingress_err2' 'koi_incl_err1' 'koi_incl_err2' 'koi_teq_err1'
 'koi_teq_err2' 'koi_sage' 'koi_sage_err1' 'koi_sage_err2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['koi_gmag_err' 'koi_rmag_err' 'koi_imag_err' 'koi_zmag_err'
 'koi_kepmag_err' 'koi_model_dof' 'koi_model_chisq' 'koi_eccen_err1'
 'koi_eccen_err2' 'koi_longp' 'koi_longp_err1' 'koi_longp_err2'
 'koi_sma_err1' 'koi_sma_err2' 'koi_ingress' 'koi_ingress_err1'
 

AttributeError: 'numpy.ndarray' object has no attribute 'isna'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.4 MB/s eta 0:00:00
